<a href="https://colab.research.google.com/github/undefinedzack/stock-market-prediction-using-sentiment-analysis/blob/master/Copy_of_mainW2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Manipulation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab_Data/
%ls

/content/drive/MyDrive/Colab_Data
stockerbot-export1.csv  tweet_sentiment.csv


In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving stockerbot-export1.csv to stockerbot-export1.csv
Saving tweet_sentiment.csv to tweet_sentiment.csv


In [ ]:
import pandas as pd 
import io
import random

In [ ]:
df = pd.read_csv('stockerbot-export1.csv') 
senti = pd.read_csv('tweet_sentiment.csv') 

In [ ]:
df.head(10)

,id,text,timestamp,source,symbols,company_names,url,verified
0,1.0197E+18,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,TRUE
1,1.01971E+18,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,TRUE
2,1.01971E+18,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,TRUE
3,1.01972E+18,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,TRUE
4,1.01972E+18,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,TRUE
5,1.01972E+18,.@RealMoney's David Butler's favorite FANG sto...,Wed Jul 18 23:04:00 +0000 2018,jimcramer,FB-GOOGL-GOOG,Facebook*Alphabet*Alphabet,http://bit.ly/2NrYxje,TRUE
6,1.01972E+18,Don’t miss my convo with one of my favorite th...,Wed Jul 18 23:06:58 +0000 2018,ianbremmer,HRS,Harris,https://twitter.com/samharrisorg/status/101971...,TRUE
7,1.01972E+18,U.S. intelligence documents on Nelson Mandela ...,Wed Jul 18 23:08:45 +0000 2018,Reuters,INTC-USB,Intel*U.S.,https://reut.rs/2O0ypNf,TRUE
8,1.01972E+18,Senate wants emergency alerts to go out throug...,Wed Jul 18 23:09:00 +0000 2018,TechCrunch,NFLX,Netflix,https://tcrn.ch/2L8DsgT,TRUE
9,1.01972E+18,Hedge fund manager Marc Larsy says bitcoin $40...,Wed Jul 18 23:10:41 +0000 2018,MarketWatch,BTC,Bitcoin,https://on.mktw.net/2Ntr7k9,TRUE


In [ ]:
df['text'][1]

"The price of lumber $LB_F is down 22% since hitting its YTD highs. The Macy's $M turnaround is still happening.… https://t.co/XnKsV4De39"

In [ ]:
len(df['text'])

28440

In [ ]:
##regular expressions for ids and links from text section
pat1= r'@[A-Za-z0-9]+'
pat2= r'https?://[A-Za-z0-9./]+'
#pat1|pat2
combined_pat=r'|'.join((pat1,pat2))
#no word should start with special symbol
pat3= r'[^a-zA-Z]'
combined_pat2=r'|'.join((combined_pat,pat3))

In [ ]:
#demonstration
import re
re.sub(combined_pat2,' ','The price of lumber $LB_F is down 22% since hitting its YTD highs. The Macys $M turnaround is still happening.… https://t.co/XnKsV4De39')

'The price of lumber  LB F is down     since hitting its YTD highs  The Macys  M turnaround is still happening    '

In [ ]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# nltk.download("popular")
# ps= PorterStemmer()
cleaned_tweets = []

for i in range(0,len(df['text'])):
    tweets = re.sub(combined_pat2,' ',df['text'][i])
    tweets=tweets.lower()
    tweets=tweets.split()
    # tweets = [ps.stem(word) for word in tweets if not word in set(stopwords.words('english'))]
    tweets = ' '.join(tweets)
    cleaned_tweets.append(tweets)
df['cleaned_tweets']=cleaned_tweets

In [ ]:
df['cleaned_tweets'].head(10)

0    video i was in my office i was minding my own ...
1    the price of lumber lb f is down since hitting...
2                  who says the american dream is dead
3    barry silbert is extremely optimistic on bitco...
4    how satellites avoid attacks and space junk wh...
5    s david butler s favorite fang stock isn t rea...
6    don t miss my convo with one of my favorite th...
7    u s intelligence documents on nelson mandela m...
8    senate wants emergency alerts to go out throug...
9    hedge fund manager marc larsy says bitcoin k i...
Name: cleaned_tweets, dtype: object

# !!!Removal of Stop words!!!

In [ ]:
# from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
# removed_stopwords_tweets = []

# for tweet in df['cleaned_tweets']:
#   temp = remove_stopwords(tweet)
#   removed_stopwords_tweets.append(temp)

# Tokenization

In [ ]:
from gensim.utils import simple_preprocess

In [ ]:
senti.head(5)

,cleaned_tweets,sentiment
0,video offic mind busi david solomon tell gs in...,0
1,price lumber lb f sinc hit ytd high maci turna...,0
2,say american dream dead,-1
3,barri silbert extrem optimist bitcoin predict ...,1
4,satellit avoid attack space junk circl earth paid,-1


In [ ]:
import numpy as np

In [ ]:
import gensim
from gensim.models.word2vec import Word2Vec

#from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
#tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split

In [ ]:
senti.head(10)

,cleaned_tweets,sentiment
0,video offic mind busi david solomon tell gs in...,0
1,price lumber lb f sinc hit ytd high maci turna...,0
2,say american dream dead,-1
3,barri silbert extrem optimist bitcoin predict ...,1
4,satellit avoid attack space junk circl earth paid,-1
5,david butler favorit fang stock realmoneysod a...,0
6,miss convo one favorit thinker,0
7,u intellig document nelson mandela made public,0
8,senat want emerg alert go netflix spotifi etc,1
9,hedg fund manag marc larsi say bitcoin k possibl,0


In [ ]:
senti['tokenized_text'] = [simple_preprocess(str(tweet), deacc=True) for tweet in senti['cleaned_tweets']]
senti.head(10)

,cleaned_tweets,sentiment,tokenized_text
0,video offic mind busi david solomon tell gs in...,0,"[video, offic, mind, busi, david, solomon, tel..."
1,price lumber lb f sinc hit ytd high maci turna...,0,"[price, lumber, lb, sinc, hit, ytd, high, maci..."
2,say american dream dead,-1,"[say, american, dream, dead]"
3,barri silbert extrem optimist bitcoin predict ...,1,"[barri, silbert, extrem, optimist, bitcoin, pr..."
4,satellit avoid attack space junk circl earth paid,-1,"[satellit, avoid, attack, space, junk, circl, ..."
5,david butler favorit fang stock realmoneysod a...,0,"[david, butler, favorit, fang, stock, realmone..."
6,miss convo one favorit thinker,0,"[miss, convo, one, favorit, thinker]"
7,u intellig document nelson mandela made public,0,"[intellig, document, nelson, mandela, made, pu..."
8,senat want emerg alert go netflix spotifi etc,1,"[senat, want, emerg, alert, go, netflix, spoti..."
9,hedg fund manag marc larsi say bitcoin k possibl,0,"[hedg, fund, manag, marc, larsi, say, bitcoin,..."


#Splitting into test and training

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    senti['tokenized_text'],
    senti['sentiment'],
    shuffle = True,
    test_size = 0.3,
    random_state = random.randint(1,100)
)

In [ ]:
x_train[0]

['video',
 'offic',
 'mind',
 'busi',
 'david',
 'solomon',
 'tell',
 'gs',
 'intern',
 'learn',
 'wa']

# Word2Vec Model

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
model = Word2Vec(sentences=x_train,size=200, min_count=10)
model.init_sims(replace=True)

model_name="200features_10minwords"
model.save(model_name)


2021-02-19 10:51:52,445 : INFO : collecting all words and their counts
2021-02-19 10:51:52,445 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-19 10:51:52,486 : INFO : PROGRESS: at sentence #10000, processed 101195 words, keeping 9840 word types
2021-02-19 10:51:52,524 : INFO : collected 12693 word types from a corpus of 206098 raw words and 19908 sentences
2021-02-19 10:51:52,525 : INFO : Loading a fresh vocabulary
2021-02-19 10:51:52,540 : INFO : effective_min_count=10 retains 2909 unique words (22% of original 12693, drops 9784)
2021-02-19 10:51:52,541 : INFO : effective_min_count=10 leaves 180984 word corpus (87% of original 206098, drops 25114)
2021-02-19 10:51:52,559 : INFO : deleting the raw counts dictionary of 12693 items
2021-02-19 10:51:52,560 : INFO : sample=0.001 downsamples 48 most-common words
2021-02-19 10:51:52,561 : INFO : downsampling leaves estimated 165104 word corpus (91.2% of prior 180984)
2021-02-19 10:51:52,575 : INFO : estima

In [ ]:
model.wv.vectors.shape

(2909, 200)

In [ ]:
model.wv.most_similar("low")

[('today', 0.9388736486434937),
 ('swing', 0.9308421611785889),
 ('room', 0.9253183603286743),
 ('trade', 0.9223634600639343),
 ('costba', 0.9200249314308167),
 ('dgb', 0.9179794788360596),
 ('come', 0.9113341569900513),
 ('huge', 0.9102002382278442),
 ('rt', 0.908700168132782),
 ('visit', 0.90758216381073)]

In [ ]:
model.wv['high']

array([ 9.33943391e-02,  1.83628257e-02, -7.31661767e-02,  2.20122151e-02,
        9.81719419e-02,  7.57480320e-03, -8.98958668e-02, -5.12202531e-02,
       -1.20134234e-01, -2.62328777e-02,  1.01101212e-01, -2.08177809e-02,
       -1.33806765e-01, -2.32568737e-02,  5.95696010e-02, -7.97658321e-03,
        2.41021663e-02, -3.02576926e-02, -3.62009965e-02, -3.08113284e-02,
       -3.13441679e-02,  2.28232500e-04, -5.26359417e-02,  4.85615022e-02,
       -5.94599843e-02, -1.05895754e-03, -2.60271151e-02,  7.98697248e-02,
        3.80708016e-02,  2.34661996e-02,  4.20949906e-02,  2.77988762e-02,
       -2.60840878e-02, -2.18718033e-02,  2.49187276e-02, -4.74505834e-02,
       -4.21500988e-02, -1.15665831e-01, -2.68791988e-02,  3.24094817e-02,
        5.67540675e-02, -1.25736699e-01,  9.17347372e-02, -2.84599345e-02,
       -3.37998345e-02, -1.87700599e-01,  6.46536797e-02, -5.34543209e-02,
        1.57552078e-01,  4.27442193e-02,  3.25426646e-02, -5.28507009e-02,
       -3.67673486e-02, -

#Combining Word Vectors of each tweet

In [ ]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [ ]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

combining for x_train vectors

In [ ]:
# Calculating average feature vector for training set
 
trainDataVecs = getAvgFeatureVecs(x_train, model, 200)

Review 0 of 19908


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 1000 of 19908
Review 2000 of 19908
Review 3000 of 19908
Review 4000 of 19908
Review 5000 of 19908
Review 6000 of 19908


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Review 7000 of 19908
Review 8000 of 19908
Review 9000 of 19908
Review 10000 of 19908
Review 11000 of 19908
Review 12000 of 19908
Review 13000 of 19908
Review 14000 of 19908
Review 15000 of 19908
Review 16000 of 19908
Review 17000 of 19908
Review 18000 of 19908
Review 19000 of 19908


combining of x_test vectors

In [ ]:
testDataVecs = getAvgFeatureVecs(x_test, model, 200)

Review 0 of 8532


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Review 1000 of 8532
Review 2000 of 8532
Review 3000 of 8532
Review 4000 of 8532
Review 5000 of 8532
Review 6000 of 8532
Review 7000 of 8532
Review 8000 of 8532


In [ ]:
print(np.any(np.isnan(trainDataVecs)))
print(np.all(np.isfinite(trainDataVecs)))
trainDataVecs = np.nan_to_num(trainDataVecs,nan=0)

True
False


#Fitting

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier()

clf.fit(trainDataVecs,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

#Testing

In [ ]:
testDataVecs = np.nan_to_num(testDataVecs,nan=0)

In [ ]:
from sklearn.metrics import classification_report
test_predictions=clf.predict(testDataVecs)
print(classification_report(y_test,test_predictions))

              precision    recall  f1-score   support

          -1       0.47      0.52      0.50       774
           0       0.83      0.81      0.82      5190
           1       0.70      0.71      0.71      2568

    accuracy                           0.76      8532
   macro avg       0.67      0.68      0.67      8532
weighted avg       0.76      0.76      0.76      8532

